In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn import preprocessing
from matplotlib import pyplot as pl
os.chdir('./Desktop/TDIChallenge')
os.getcwd()


'/Users/naeemnowrouzi/Desktop/TDIChallenge'

In [7]:
# Prepare for Merge
data_0 = pd.DataFrame(pd.read_csv('./all_inventors.csv', dtype={'application_number': np.int64}))
data_1 = pd.DataFrame(pd.read_csv('./application_data.csv', dtype={'application_number' : np.str}))
#data_2 = pd.DataFrame(pd.read_csv('./correspondence_address.csv'))

/Users/naeemnowrouzi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,8,9,17,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
data_1.application_number = data_1.application_number.astype(str)
data_0.application_number = data_0.application_number.astype(str)

In [12]:
# Merge
merged_data = pd.merge(data_0, data_1, how='outer', on='application_number')
merged_data = pd.DataFrame(merged_data)

In [ ]:
# Issued patents count
issued_patents = merged_data[merged_data.patent_number != 'None']

In [32]:
# Invention classes

# Load data on USPC Class Descriptions
uspc_class_descriptions = pd.read_csv('./uspc_class_descriptions.csv')
uspc_class_descriptions.uspc_class = uspc_class_descriptions.uspc_class.astype(str) 
#data_1.uspc_class = data_1.uspc_class.astype(str)

In [30]:
#unique_classes = merged_data.uspc_class.value_counts()
unique_classes = pd.DataFrame({'num_inventions' : merged_data.groupby('uspc_class')['application_number'].nunique()}).reset_index()

In [34]:
unique_classes.uspc_class = unique_classes.uspc_class.astype(str)
#uspc_class_descriptions.uspc_class = uspc_class_descriptions.uspc_class.astype(str)
class_cnts_desc = pd.merge(unique_classes, uspc_class_descriptions, on='uspc_class', how='left')
class_cnts_desc.to_csv('./'+'class_cnts_desc.csv')

In [42]:
das = class_cnts_desc.groupby('class_description').num_inventions.sum().reset_index().sort_values('num_inventions', ascending=False).reset_index()
das = das[['class_description','num_inventions']]
das.index += 1
das['ratio'] = das.num_inventions/das.num_inventions.sum()

In [49]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
import collections
output_notebook()

d = das.head(100)

# Convert dataframe to column data source
src = ColumnDataSource(d)
src.data.keys()



# Create a blank figure with labels
p = figure(plot_width = 1100, plot_height = 1100, 
           title = '100 Invention Classes with Largest Number of Inventions in all Patent Applications Filed with USPTO',
           x_axis_label = 'USPC Class', y_axis_label = 'Count')

p.vbar(source = src, x='index', bottom=0,top='num_inventions', width=1.3, alpha=0.9, color='pink')

# Hover Tool
h = HoverTool(tooltips = collections.OrderedDict([('Class Description', '@class_description'), 
                                                  ('Number of Inventions', '@num_inventions'), ('rank', '@index'), ('ratio', '@ratio')]))
p.add_tools(h)
p.xaxis.visible = False
show(p)


Loading BokehJS ...